In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from downcast import reduce
import calmap
import warnings
import kaleido
from plotly.io import to_image
from IPython.display import Image
import pickle


warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
prices = pd.read_pickle("../data/processed/item_prices.pkl")
cal = pd.read_pickle("../data/processed/daily_calendar_with_events.pkl")

In [3]:
sales_eval = pd.read_pickle('../data/processed/item_sales.pkl') 

#Criando os 28 dias para previsão
#for day in range(1914,1942):
#    sales_eval['d_' + str(day)] = 0
#    sales_eval['d_' + str(day)] = sales_eval['d_' + str(day)].astype(np.int16)

sales_eval.head()

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [4]:
# Passar da Wide form para Long form
data = sales_eval.melt(id_vars=['id','item', 'category', 'department', 'store_code', 'region'], value_vars=[i for i in sales_eval if i.startswith('d_')], var_name=['d'], value_name='sales')

data.head()

,id,item,category,department,store_code,region,d,sales
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0


In [5]:
prices['yearweek'] = prices['yearweek'].astype('Int64')
for i in range(len(prices)):
   if pd.isnull(prices['yearweek'][i]):
       prices['yearweek'][i]=prices['yearweek'][i-1]+1

In [50]:
prices.dtypes

item           string
category       string
store_code     string
yearweek       string
sell_price    float16
dtype: object

In [6]:
prices['item'] = prices['item'].astype('string')
prices['category'] = prices['category'].astype('string')
prices['store_code'] = prices['store_code'].astype('string')
prices['yearweek'] = prices['yearweek'].astype('Int64').astype('string')

In [7]:
data['item'] = data['item'].astype('string')
data['category'] = data['category'].astype('string')
data['store_code'] = data['store_code'].astype('string')


In [9]:
data.head()

,id,item,category,department,store_code,region,d,sales
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0


In [8]:
merged = pd.merge(data, cal, on='d', how='left')

In [11]:
merged.head()

,id,item,category,department,store_code,region,d,sales,date,weekday,weekday_int,event
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN


In [9]:
merged['date'] = pd.to_datetime(merged['date'])
merged['year'] = merged['date'].dt.year
merged['month'] = merged['date'].dt.month
merged['day'] = merged['date'].dt.day
merged['week'] = merged['date'].dt.isocalendar().week.apply(str)
merged['week'] = merged['week'].apply(lambda x: x.zfill(2))
merged['yearweek'] = merged['year'].apply(str) + merged['week']


In [10]:
merged2 = pd.merge(merged, prices, on=['store_code','item','yearweek'], how='left')

In [11]:
merged2=merged2.drop(['category_y'],axis=1)

In [12]:
merged2 = merged2.rename(columns={'category_x':'category'})

In [13]:
data = merged2

In [14]:
data.head()

,id,item,category,department,store_code,region,d,sales,date,weekday,weekday_int,event,year,month,day,week,yearweek,sell_price
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN,2011,1,29,04,201104,NaN
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN,2011,1,29,04,201104,NaN
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN,2011,1,29,04,201104,NaN
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN,2011,1,29,04,201104,NaN
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN,2011,1,29,04,201104,NaN


In [15]:
data["sell_price"].fillna(data.groupby("id")["sell_price"].transform("mean"), inplace=True)
data['d'] = data['d'].apply(lambda a: a.split('_')[1]).astype(np.int16)
data.drop(columns=["date","weekday"], inplace=True)
data.head()

,id,item,category,department,store_code,region,d,sales,weekday_int,event,year,month,day,week,yearweek,sell_price
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,NYC_1,New York,1,0,1,NaN,2011,1,29,04,201104,11.007812
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,NYC_1,New York,1,0,1,NaN,2011,1,29,04,201104,5.281250
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,NYC_1,New York,1,0,1,NaN,2011,1,29,04,201104,3.949219
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,NYC_1,New York,1,0,1,NaN,2011,1,29,04,201104,6.019531
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,NYC_1,New York,1,0,1,NaN,2011,1,29,04,201104,3.914062


In [16]:
import pickle
data.to_pickle('../data/processed/fe1_data.pkl')

In [2]:
import pickle
data = pd.read_pickle("../data/processed/fe1_data.pkl")

### Label Encoding

In [3]:
cols = data.dtypes.index.tolist()
d_types = data.dtypes.values.tolist()

for i,type in enumerate(d_types):
    if type.name == 'category':
        data[cols[i]] = data[cols[i]].cat.codes

In [4]:
#Intoducing lags and rolling features
#lag features

lags = [1,2,3,5,7,14,21,28]
for lag in lags:
    data["lag_" + str(lag)] = data.groupby("id")["sales"].shift(lag).astype(np.float16)

In [5]:
#rolling mean features

data['rolling_mean_10'] = data.groupby('id')['sales'].transform(lambda x: x.rolling(10).mean())
data['rolling_mean_20'] = data.groupby('id')['sales'].transform(lambda x: x.rolling(20).mean())
data['rolling_mean_30'] = data.groupby('id')['sales'].transform(lambda x: x.rolling(30).mean())

In [40]:
data['event'].unique()

array([-1,  3,  2,  4,  1,  0], dtype=int8)

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58327370 entries, 0 to 58327369
Data columns (total 27 columns):
 #   Column           Dtype  
---  ------           -----  
 0   id               int16  
 1   item             string 
 2   category         string 
 3   department       int8   
 4   store_code       string 
 5   region           int8   
 6   d                int16  
 7   sales            int16  
 8   weekday_int      int8   
 9   event            int8   
 10  year             int64  
 11  month            int64  
 12  day              int64  
 13  week             object 
 14  yearweek         object 
 15  sell_price       float16
 16  lag_1            float16
 17  lag_2            float16
 18  lag_3            float16
 19  lag_5            float16
 20  lag_7            float16
 21  lag_14           float16
 22  lag_21           float16
 23  lag_28           float16
 24  rolling_mean_10  float64
 25  rolling_mean_20  float64
 26  rolling_mean_30  float64
dtypes: float16

> Uma vez que o tamanho da base de dados é superior a 6.7GB, então iremos considerar uma amostra da presente base de dados.
Vamos considerar apenas os últimos 914 dias.

In [6]:

data = data[data['d']>1000]
data.head()

,id,item,category,department,store_code,region,d,sales,weekday_int,event,...,lag_2,lag_3,lag_5,lag_7,lag_14,lag_21,lag_28,rolling_mean_10,rolling_mean_20,rolling_mean_30
30490000,3,ACCESORIES_1_001,ACCESORIES,0,NYC_1,1,1001,2,7,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.3,0.40,0.433333
30490001,13,ACCESORIES_1_002,ACCESORIES,0,NYC_1,1,1001,0,7,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.10,0.200000
30490002,23,ACCESORIES_1_003,ACCESORIES,0,NYC_1,1,1001,0,7,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000
30490003,33,ACCESORIES_1_004,ACCESORIES,0,NYC_1,1,1001,0,7,-1,...,2.0,1.0,2.0,1.0,0.0,3.0,0.0,1.7,1.70,1.800000
30490004,43,ACCESORIES_1_005,ACCESORIES,0,NYC_1,1,1001,1,7,-1,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.05,1.033333


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27837370 entries, 30490000 to 58327369
Data columns (total 27 columns):
 #   Column           Dtype  
---  ------           -----  
 0   id               int16  
 1   item             string 
 2   category         string 
 3   department       int8   
 4   store_code       string 
 5   region           int8   
 6   d                int16  
 7   sales            int16  
 8   weekday_int      int8   
 9   event            int8   
 10  year             int64  
 11  month            int64  
 12  day              int64  
 13  week             object 
 14  yearweek         object 
 15  sell_price       float16
 16  lag_1            float16
 17  lag_2            float16
 18  lag_3            float16
 19  lag_5            float16
 20  lag_7            float16
 21  lag_14           float16
 22  lag_21           float16
 23  lag_28           float16
 24  rolling_mean_10  float64
 25  rolling_mean_20  float64
 26  rolling_mean_30  float64
dtypes: 

> Repare-se que ao considerarmos uma amostra o ficheiro ficou bem mais leve , 3.2GB.

In [7]:
data.to_pickle('../data/processed/fe2_data.pkl')

> Agora que realizamos os merges de todos os preços e dados das datas e introduzimos features de vendas anteriores como os lags e médias móveis estamos prontos para o estágio da modelação.